In [0]:
import requests
from datetime import datetime
import logging
import urllib3

import sailpoint
import sailpoint.v2024
from sailpoint.configuration import (Configuration, ConfigurationParams)
from sailpoint.paginator import Paginator

from pyspark.sql.functions import lit
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, TimestampType, MapType, DateType

In [0]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create console handler and set level to info
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

# Create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Add formatter to ch
ch.setFormatter(formatter)

# Add ch to logger
logger.addHandler(ch)

In [0]:
tenant = 'devrel-ga-5668'
client_name = 'sailpoint-readall-client'
secret_name = 'sailpoint-readall-secret'

In [0]:
logger.info('Defining API credentials')
api_url = f'https://{tenant}.api.identitynow-demo.com'
token_url = f'https://{tenant}.api.identitynow-demo.com/oauth/token'
client_id = dbutils.secrets.get(scope='sailpoint-devrel', key=client_name)
client_secret = dbutils.secrets.get(scope='sailpoint-devrel', key=secret_name)

In [0]:
logger.info('Configuring SailPoint SDK')
configurationParams = ConfigurationParams()
configurationParams.base_url = api_url
configurationParams.token_url = token_url
configurationParams.client_id = client_id
configurationParams.client_secret = client_secret
configuration = Configuration(configurationParams)
configuration.experimental = True
configuration.retries = urllib3.Retry(
    total=5,  # Total number of retries
    backoff_factor=10,  # Backoff factor for retry delay
    status_forcelist=[429, 500, 502, 503, 504],  # HTTP status codes to retry on
    allowed_methods=["GET"]  # HTTP methods to retry
)

In [0]:
async def get_total_account_count(filter):
    logger.info('Retrieving total identity count...')
    try:
        with sailpoint.v2024.ApiClient(configuration) as api_client:
            api_instance = sailpoint.v2024.AccountsApi(api_client)
            response = api_instance.list_accounts_with_http_info(limit=1, count=True, filters=filter)
            return response.headers['X-Total-Count']
    except Exception as e:
        raise Exception(f'Error Retrieving total account count: {e}')

async def get_accounts(total_identity_count, filter):
    logger.info('Retrieving accounts...')
    try:
        all_accounts = []
        with sailpoint.v2024.ApiClient(configuration) as api_client:
            api_instance = sailpoint.v2024.AccountsApi(api_client)
            accounts = Paginator.paginate(
                api_instance.list_accounts,
                result_limit=total_account_count,
                limit=250,
                sorters='modified',
                filters=filter
            )
            logger.info(f'Example account: {accounts[0]}')
            return accounts
            logger.info(f'Retrieved {len(accounts)} accounts')
            return accounts
    except Exception as e:
        logger.error(f'Error Retrieving accounts: {e}')
        raise Exception(e)

In [0]:
total_account_count = int(await get_total_account_count(None))
logger.info(f'Total Accounts: {total_account_count}')
all_accounts = await get_accounts(total_account_count, None)
logger.info(f'Total Retrieved Accounts: {len(all_accounts)}')

In [0]:
object_schema = StructType(
    [
        StructField("id", StringType(), True),
        StructField("name", StringType(), True),
        StructField("type", StringType(), True),
    ]
)

account_schema = StructType([
    StructField("id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("created", TimestampType(), True),
    StructField("modified", TimestampType(), True),
    StructField("source_id", StringType(), True),
    StructField("source_name", StringType(), True),
    StructField("identity_id", StringType(), True),
    StructField("cloud_lifecycle_state", StringType(), True),
    StructField("identity_state", StringType(), True),
    StructField("connection_type", StringType(), True),
    StructField("is_machine", BooleanType(), True),
    StructField("attributes", MapType(StringType(), StringType()), True),
    StructField("authoritative", BooleanType(), True),
    StructField("description", StringType(), True),
    StructField("disabled", BooleanType(), True),
    StructField("locked", BooleanType(), True),
    StructField("native_identity", StringType(), True),
    StructField("system_account", BooleanType(), True),
    StructField("uncorrelated", BooleanType(), True),
    StructField("uuid", StringType(), True),
    StructField("manually_correlated", BooleanType(), True),
    StructField("has_entitlements", BooleanType(), True),
    StructField("identity", object_schema, True),
    StructField("source_owner", object_schema, True),
    StructField("features", StringType(), True),
    StructField("origin", StringType(), True),
    StructField("owner_identity", object_schema, True)
])

In [0]:
try:
    logger.info('Creating data frame...')
    accounts_df = spark.createDataFrame(all_accounts, account_schema)
    today = datetime.utcnow().date()
    accounts_df = accounts_df.withColumn('day', lit(today))

    logger.info('Writing to delta...')
    accounts_df.write \
        .format('delta') \
        .mode('append') \
        .partitionBy('day') \
        .saveAsTable(f'`dunker_databricks_space`.sailpoint.accounts')

except Exception as e:
    logger.error(f'Error in creating data frame or writing to delta: {e}')